# PyMapAccuracy Tutorial: Reproducing Published Paper Examples

This notebook demonstrates the **PyMapAccuracy** package by reproducing the exact numerical examples from the original research papers. This ensures you can replicate published results and understand the methodology.

## 📚 Paper Examples Covered

1. **Stehman (2014)**: "Estimating area and map accuracy for stratified random sampling when the strata are different from the map classes"
   - *International Journal of Remote Sensing*, 35(13), 4923-4943
   - Used when sampling strata ≠ map classes (e.g., reusing reference data from another map)

2. **Olofsson et al. (2013)**: "Making better use of accuracy data in land change studies"
   - *Remote Sensing of Environment*, 129, 122-131
   - Used when map classes = sampling strata (most common case)

In [3]:
# Import required libraries
import matplotlib.pyplot as plt
import pandas as pd

# Import PyMapAccuracy estimators
from pymapaccuracy.estimators import olofsson, stehman2014

# Set up plotting style for publication-quality figures
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Set pandas display options
pd.set_option('display.precision', 6)
pd.set_option('display.max_columns', None)

## 📋 Example 1: Stehman (2014) - When Strata ≠ Map Classes

### Paper Context
**Citation**: Stehman, S. V. (2014). Estimating area and map accuracy for stratified random sampling when the strata are different from the map classes. *International Journal of Remote Sensing*, 35(13), 4923-4943.

### Numerical Example Setup
- **Sampling Strata**: 4 classes from map used for stratification (A, B, C, D) 
- **Map Classes**: 4 classes from map being assessed (A, B, C, D)
- **Sample Size**: 10 samples per stratum (40 total)

In [5]:
# Stehman (2014) Paper Example - Exact Replication

# Sampling strata: Administrative regions (10 samples each)
strata =  ["A"] * 10 + ["B"] * 10 + ["C"] * 10 + ["D"] * 10

# Map predictions: Land cover classes from the map being evaluated
map_predictions = (
        ["A"] * 7
        + ["B"] * 3
        + ["A"] * 1
        + ["B"] * 11
        + ["C"] * 6
        + ["B"] * 2
        + ["D"] * 10
    )

# Reference (ground truth): True land cover from field verification
reference = (
        ["A"] * 5
        + ["C"] * 1
        + ["B"] * 1
        + ["A"] * 1
        + ["B"] * 1
        + ["C"] * 1  # Stratum A (10)
        + ["A"] * 1
        + ["B"] * 5
        + ["A"] * 2
        + ["B"] * 2  # Stratum B (10)
        + ["C"] * 5
        + ["D"] * 2
        + ["B"] * 2
        + ["A"] * 1  # Stratum C (10)
        + ["D"] * 7
        + ["C"] * 2
        + ["B"] * 1
    )

# Area of each stratum (in any consistent unit - the paper doesn't specify, but we'll assume hectares)
stratum_areas = {
    "A": 40000,  # Administrative region A covers 40,000 hectares
    "B": 30000,  # Administrative region B covers 30,000 hectares
    "C": 20000,  # Administrative region C covers 20,000 hectares
    "D": 10000   # Administrative region D covers 10,000 hectares
}

print(f"   • Total samples: {len(strata)}")
print(f"   • Sampling strata: {sorted(set(strata))}")
print(f"   • Map classes: {sorted(set(map_predictions))}")
print(f"   • Reference classes: {sorted(set(reference))}")
print(f"   • Total study area: {sum(stratum_areas.values()):,} hectares")

# Create summary table
sample_summary = pd.DataFrame({
    'Stratum': ['A', 'B', 'C', 'D'],
    'Samples': [10, 10, 10, 10],
    'Area (ha)': [40000, 30000, 20000, 10000],
    'Area (%)': [40.0, 30.0, 20.0, 10.0]
})
print("\n Sampling Design Summary: \n")
print(sample_summary.to_string(index=False))

   • Total samples: 40
   • Sampling strata: ['A', 'B', 'C', 'D']
   • Map classes: ['A', 'B', 'C', 'D']
   • Reference classes: ['A', 'B', 'C', 'D']
   • Total study area: 100,000 hectares

 Sampling Design Summary: 

Stratum  Samples  Area (ha)  Area (%)
      A       10      40000      40.0
      B       10      30000      30.0
      C       10      20000      20.0
      D       10      10000      10.0


In [6]:
# Run the Stehman (2014) estimator
results_stehman = stehman2014(strata, reference, map_predictions, stratum_areas)
order = ['A', 'B', 'C', 'D']
print("Stehman (2014) Results:")
print("=" * 50)
print(f"Overall Accuracy: {results_stehman['OA']:.4f}")
print(f"Standard Error OA: {results_stehman['SEoa']:.4f}")
print()
print("User's Accuracy:")
for cls in order:
    if cls in results_stehman['UA'].index:
        print(f"  {cls}: {results_stehman['UA'][cls]:.4f}")
print()
print("Producer's Accuracy:")
for cls in order:
    if cls in results_stehman['PA'].index:
        print(f"  {cls}: {results_stehman['PA'][cls]:.4f}")
print()
print("Area Estimates (proportions):")
for cls in order:
    if cls in results_stehman['area'].index:
        print(f"  {cls}: {results_stehman['area'][cls]:.4f}")
print()
print("Standard Errors for Area:")
for cls in order:
    if cls in results_stehman['SEa'].index:
        print(f"  {cls}: {results_stehman['SEa'][cls]:.4f}")

Stehman (2014) Results:
Overall Accuracy: 0.6300
Standard Error OA: 0.0846

User's Accuracy:
  A: 0.7419
  B: 0.5745
  C: 0.5000
  D: 0.7000

Producer's Accuracy:
  A: 0.6571
  B: 0.7941
  C: 0.3000
  D: 0.6364

Area Estimates (proportions):
  A: 0.3500
  B: 0.3400
  C: 0.2000
  D: 0.1100

Standard Errors for Area:
  A: 0.0822
  B: 0.0759
  C: 0.0643
  D: 0.0307


### 📈 Comparison of PyMapAccuracy with Results from Paper

In [ ]:
# The following values are taken from Stehman (2014) paper for comparison
# Proportion of area of class A in paper = 0.35
paper_proportion_A = 0.35
estimated_proportion_A = results_stehman['area']['A'] if 'A' in results_stehman['area'].index else None

# Proportion of area of Class C in paper = 0.20
paper_proportion_C = 0.20
estimated_proportion_C = results_stehman['area']['C'] if 'C' in results_stehman['area'].index else None

# Overall accuracy in paper = 0.63
overall_accuracy_paper = 0.63

# User's accuracy for class B in paper = 0.574
users_accuracy_B_paper = 0.574
users_accuracy_B_estimated = results_stehman['UA']['B'] if 'B' in results_stehman['UA'].index else None

# Producer's accuracy for class B in paper = 0.794
producers_accuracy_B_paper = 0.794
producers_accuracy_B_estimated = results_stehman['PA']['B'] if 'B' in results_stehman['PA'].index else None

# Cell (i, j) of the error matrix, Pij (i = 2, j = 3) = 0.08
P_23_paper = 0.08
P_23_estimated = results_stehman['matrix'].loc['B', 'C'] if ('B' in results_stehman['matrix'].index and 'C' in results_stehman['matrix'].columns) else None

# Standard error (SE) for proportion of area of class A = 0.082
SE_area_A_paper = 0.082
SE_area_A_estimated = results_stehman['SEa']['A'] if 'A' in results_stehman['SEa'].index else None

# SE for proportion of area of class C = 0.064
SE_area_C_paper = 0.064
SE_area_C_estimated = results_stehman['SEa']['C'] if 'C' in results_stehman['SEa'].index else None

# SE for overall accuracy = 0.085
SE_OA_paper = 0.085
SE_OA_estimated = results_stehman['SEoa']

# SE for user’s accuracy of class B = 0.125
SE_UA_B_paper = 0.125
SE_UA_B_estimated = results_stehman['SEua']['B'] if 'B' in results_stehman['SEua'].index else None

# SE for producer’s accuracy of class B = 0.114
SE_PA_B_paper = 0.114
SE_PA_B_estimated = results_stehman['SEpa']['B'] if 'B' in results_stehman['SEpa'].index else None

comparison_data = [
    ['Proportion of area (A)', paper_proportion_A, estimated_proportion_A],
    ['Proportion of area (C)', paper_proportion_C, estimated_proportion_C],
    ['Overall Accuracy', overall_accuracy_paper, results_stehman['OA']],
    ["User's Accuracy (B)", users_accuracy_B_paper, users_accuracy_B_estimated],
    ["Producer's Accuracy (B)", producers_accuracy_B_paper, producers_accuracy_B_estimated],
    ['Error Matrix P(B,C)', P_23_paper, P_23_estimated],
    ['SE Area (A)', SE_area_A_paper, SE_area_A_estimated],
    ['SE Area (C)', SE_area_C_paper, SE_area_C_estimated],
    ['SE Overall Accuracy', SE_OA_paper, SE_OA_estimated],
    ["SE User's Accuracy (B)", SE_UA_B_paper, SE_UA_B_estimated],
    ["SE Producer's Accuracy (B)", SE_PA_B_paper, SE_PA_B_estimated]
]

df_comparison = pd.DataFrame(comparison_data, columns=['Metric', 'Paper Value', 'PyMapAccuracy Value'])

print("\n📋 Comparison of Key Metrics:")
print("Please note that due to rounding differences, some values may differ slightly \n")
# make sure entire table is printed
print(df_comparison.to_string(index=False))





📋 Comparison of Key Metrics:
Please note that due to rounding differences, some values may differ slightly 

                    Metric  Paper Value  PyMapAccuracy Value
    Proportion of area (A)        0.350             0.350000
    Proportion of area (C)        0.200             0.200000
          Overall Accuracy        0.630             0.630000
       User's Accuracy (B)        0.574             0.574468
   Producer's Accuracy (B)        0.794             0.794118
       Error Matrix P(B,C)        0.080             0.080000
               SE Area (A)        0.082             0.082248
               SE Area (C)        0.064             0.064280
       SE Overall Accuracy        0.085             0.084642
    SE User's Accuracy (B)        0.125             0.124782
SE Producer's Accuracy (B)        0.114             0.116548


## 📋 Example 2: Olofsson et al. (2013) - When Map Classes = Strata

### Paper Context
**Citation**: Olofsson, P., Foody, G. M., Stehman, S. V., & Woodcock, C. E. (2013). Making better use of accuracy data in land change studies: Estimating accuracy and area and quantifying uncertainty using stratified estimation. *Remote Sensing of Environment*, 129, 122-131.

### Example 1 Setup (from paper)
- **Map Classes**: 1=Stable forest, 2=Forest loss, 3=Non-forest
- **Scenario**: Land cover change mapping study
- **Sample Design**: Stratified random sampling with map classes as strata

## 2. Olofsson et al. (2013) Example

The Olofsson et al. (2013) method is designed for area estimation with bias correction from map-based stratified sampling. This approach uses map classes as sampling strata and provides bias-corrected area estimates.

### Example 1 from Olofsson et al. (2013) Paper

This replicates the exact Example 1 from the paper with:
- **Classes**: 1=Stable forest, 2=Forest loss, 3=Non-forest  
- **Scenario**: Land cover mapping where map classes serve as sampling strata
- **Sample size**: 500 samples total
- **Study area**: 1,755,124 pixels

In [10]:
# Olofsson et al. (2013) Example 1 - exact data from test_olofsson_example1_2013
"""Replicates Example 1 from Olofsson et al. (2013)."""

r = ["1"] * 102 + ["2"] * 280 + ["3"] * 118
m = (
    ["1"] * 97
    + ["2"] * 3
    + ["3"] * 2
    + ["2"] * 279
    + ["3"] * 1
    + ["1"] * 3
    + ["2"] * 18
    + ["3"] * 97
)
Nh = {"1": 22353, "2": 1122543, "3": 610228}
total_area_pixels = sum(Nh.values())

In [ ]:
# Run Olofsson et al. (2013) estimator
results_olofsson = olofsson(r, m, Nh)

print("Olofsson et al. (2013) Results:")
print("=" * 50)
print(f"Overall Accuracy: {results_olofsson['OA']:.4f}")
print(f"Standard Error OA: {results_olofsson['SEoa']:.4f}")
print()
print("User's Accuracy:")
for cls in ["1", "2", "3"]:
    if cls in results_olofsson['UA'].index:
        print(f"  Class {cls}: {results_olofsson['UA'][cls]:.4f}")
print()
print("Producer's Accuracy:")
for cls in ["1", "2", "3"]:
    if cls in results_olofsson['PA'].index:
        print(f"  Class {cls}: {results_olofsson['PA'][cls]:.4f}")
print()
print("Area Estimates (proportions):")
for cls in ["1", "2", "3"]:
    if cls in results_olofsson['area'].index:
        print(f"  Class {cls}: {results_olofsson['area'][cls]:.4f}")
print()
print("Standard Errors for Area:")
for cls in ["1", "2", "3"]:
    if cls in results_olofsson['SEa'].index:
        print(f"  Class {cls}: {results_olofsson['SEa'][cls]:.4f}")

# Expected results from the paper for validation
print()
print("Expected results from paper:")
print(f"  Area Class 1: {0.02570326:.5f} (proportion)")
print("  Overall Accuracy: ~0.9444")
print("  User's Accuracy Class 1: 0.97")
print("  Producer's Accuracy Class 1: ~0.4806")

class order  ['1', '2', '3']
Olofsson et al. (2013) Results:
Overall Accuracy: 0.9444
Standard Error OA: 0.0112

User's Accuracy:
  Class 1: 0.9700
  Class 2: 0.9300
  Class 3: 0.9700

Producer's Accuracy:
  Class 1: 0.4806
  Class 2: 0.9942
  Class 3: 0.8969

Area Estimates (proportions):
  Class 1: 0.0257
  Class 2: 0.5983
  Class 3: 0.3760

Standard Errors for Area:
  Class 1: 0.0061
  Class 2: 0.0101
  Class 3: 0.0106

Expected results from paper:
  Area Class 1: 0.02570 (proportion)
  Overall Accuracy: ~0.9444
  User's Accuracy Class 1: 0.97
  Producer's Accuracy Class 1: ~0.4806


In [20]:
# The following values are taken from Olofsson et al. (2013) paper for comparison
# Proportion of area of class 1 in paper = 0.02570326 (Eq. 9)
paper_proportion_1 = 0.026
estimated_proportion_1 = results_olofsson['area']['1'] if '1' in results_olofsson['area'].index else None

# Area of class 1 in paper = 45651 ha (Eq. 10)
paper_area_1 = 45651
estimated_area_1 = results_olofsson['area']['1'] * total_area_pixels if '1' in results_olofsson['area'].index else None

# Standard error (SE) for proportion of area of class 1 = 0.00613 (Eq. 11)
SE_proportion_1_paper = 0.00613
SE_proportion_1_estimated = results_olofsson['SEa']['1'] if '1' in results_olofsson['SEa'].index else None

# Standard error (SE) for the adjusted area of class 1 = 10751 ha (Eq. 12)
SE_area_1_paper = 10751
SE_area_1_estimated = results_olofsson['SEa']['1'] * total_area_pixels if '1' in results_olofsson['SEa'].index else None

# Confidence interval (CI) for estimated area of class 1 = (+- 21502 ha) (Eq. 13)
CI_area_1_paper = 21502
CI_area_1_estimated = results_olofsson["CI_halfwidth_a"]['1'] * total_area_pixels if '1' in results_olofsson['CI_halfwidth_a'].index else None

# User's accuracy for class 1 in paper = 0.969 (Eq. 14)
users_accuracy_1_paper = 0.969
users_accuracy_1_estimated = results_olofsson['UA']['1'] if '1' in results_olofsson['UA'].index else None

# Producer's accuracy for class 1 in paper = 0.480 (Eq. 15)
producers_accuracy_1_paper = 0.480
producers_accuracy_1_estimated = results_olofsson['PA']['1'] if '1' in results_olofsson['PA'].index else None

# Overall accuracy in paper = 0.944 (Eq. 16)
overall_accuracy_paper_olofsson = 0.944
overall_accuracy_estimated = results_olofsson['OA']

comparison_data_olofsson = [
    ['Proportion of area (1)', paper_proportion_1, estimated_proportion_1],
    ['Area of class 1 (ha)', paper_area_1, estimated_area_1],
    ['SE Proportion of area (1)', SE_proportion_1_paper, SE_proportion_1_estimated],
    ['SE Area of class 1 (ha)', SE_area_1_paper, SE_area_1_estimated],
    ['CI Area of class 1 (ha)', CI_area_1_paper, CI_area_1_estimated],
    ["User's Accuracy (1)", users_accuracy_1_paper, users_accuracy_1_estimated],
    ["Producer's Accuracy (1)", producers_accuracy_1_paper, producers_accuracy_1_estimated],
    ['Overall Accuracy', overall_accuracy_paper_olofsson, overall_accuracy_estimated]
]

df_comparison_olofsson = pd.DataFrame(comparison_data_olofsson, columns=['Metric', 'Paper Value', 'PyMapAccuracy Value'])
print("\n📋 Comparison of Key Metrics:")
print("Please note that due to rounding differences, some values may differ slightly \n")
print(df_comparison_olofsson.to_string(index=False))


📋 Comparison of Key Metrics:
Please note that due to rounding differences, some values may differ slightly 

                   Metric  Paper Value  PyMapAccuracy Value
   Proportion of area (1)      0.02600             0.025703
     Area of class 1 (ha)  45651.00000         45112.400000
SE Proportion of area (1)      0.00613             0.006126
  SE Area of class 1 (ha)  10751.00000         10751.404503
  CI Area of class 1 (ha)  21502.00000         21072.365610
      User's Accuracy (1)      0.96900             0.970000
  Producer's Accuracy (1)      0.48000             0.480631
         Overall Accuracy      0.94400             0.944417
